In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets,transforms,models
import os

In [2]:
data_transform = {
    'train':transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
    ]),
     'val':transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
    ])
}

In [3]:
data_dir = 'Flowers_Classification_dataset'

image_data = {x:datasets.ImageFolder(os.path.join(data_dir,x),data_transform[x]) for x in ['train','val']}


In [5]:
dataloaders = {x:torch.utils.data.DataLoader(image_data[x], batch_size=4, shuffle=True,num_workers=4) for x in ['train','val']}

dataset_sizes = {x:len(image_data[x]) for x in ['train','val']}
print(dataset_sizes)


class_names = image_data['train'].classes
print(class_names)

{'train': 6375, 'val': 364}
['daisy', 'dandelion']


In [ ]:
model = models.resnet18(pretrained=True)

for name,param in model.named_parameters():
    if 'fc' in name:
        param.requires_grad = True
    else:
        param.requires_grad= False

LossFn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=0.001,momentum=0.9)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    for phase in ['train','val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0


        for inputs,lables in dataloaders[phase]:
            inputs = inputs.to(device)
            lables = lables.to(device)

            optimizer.zero_grad()
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _,preds = torch.max(outputs,1)
                loss = LossFn(outputs,lables)
                if phase == "train":
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item()*inputs.size(0)
            running_corrects += torch.sum(preds == lables.data)

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase,epoch_loss,epoch_acc))

print('Done')


In [10]:
torch.save(model.state_dict(), 'model.pth')

In [ ]:
from PIL import Image


model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 1000)
model.load_state_dict(torch.load('model.pth'))
model.eval()

new_model = models.resnet18(pretrained=True)
new_model.fc = nn.Linear(new_model.fc.in_features, 2) #for the current classifications

new_model.fc.weight.data = model.fc.weight.data[0:2]
new_model.fc.bias.data = model.fc.bias.data[0:2]


In [ ]:
img = Image.open('test.jpg')
preprocess = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
    ])
input_tensor = preprocess(img)
input_batch = input_tensor.unsqueeze(0)

In [17]:
with torch.no_grad():
    output = model(input_batch)
_, pred = output.max(1)

class_names = image_data['train'].classes
class_index = pred.item()
print('Predicted class index:', class_names[class_index])

Predicted class index: daisy
